In [ ]:
# Downloads audio data for augmentation
# Borrowed from openWakeWord's automatic_model_training.ipynb, accessed March 4, 2024

import datasets
import scipy
import os

import numpy as np

from pathlib import Path
from tqdm import tqdm

## Download MIR RIR data

output_dir = "./mit_rirs"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    rir_dataset = datasets.load_dataset("davidscripka/MIT_environmental_impulse_responses", split="train", streaming=True)
    # Save clips to 16-bit PCM wav files
    for row in tqdm(rir_dataset):
        name = row['audio']['path'].split('/')[-1]
        scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))

## Download noise and background audio

# Audioset Dataset (https://research.google.com/audioset/dataset/index.html)
# Download one part of the audioset .tar files, extract, and convert to 16khz
# For full-scale training, it's recommended to download the entire dataset from
# https://huggingface.co/datasets/agkphysics/AudioSet, and
# even potentially combine it with other background noise datasets (e.g., FSD50k, Freesound, etc.)

if not os.path.exists("audioset"):
    os.mkdir("audioset")

    fname = "bal_train09.tar"
    out_dir = f"audioset/{fname}"
    link = "https://huggingface.co/datasets/agkphysics/AudioSet/resolve/main/data/" + fname
    !wget -O {out_dir} {link}
    !cd audioset && tar -xvf bal_train09.tar

    output_dir = "./audioset_16k"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    # Save clips to 16-bit PCM wav files
    audioset_dataset = datasets.Dataset.from_dict({"audio": [str(i) for i in Path("audioset/audio").glob("**/*.flac")]})
    audioset_dataset = audioset_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000))
    for row in tqdm(audioset_dataset):
        name = row['audio']['path'].split('/')[-1].replace(".flac", ".wav")
        scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))

# Free Music Archive dataset
# https://github.com/mdeff/fma

output_dir = "./fma"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    fma_dataset = datasets.load_dataset("rudraml/fma", name="small", split="train", streaming=True)
    fma_dataset = iter(fma_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000)))

    # Save clips to 16-bit PCM wav files
    n_hours = 1  # use only 1 hour of clips for this example notebook, recommend increasing for full-scale training
    for i in tqdm(range(n_hours*3600//30)):  # this works because the FMA dataset is all 30 second clips
        row = next(fma_dataset)
        name = row['audio']['path'].split('/')[-1].replace(".mp3", ".wav")
        scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))
        i += 1
        if i == n_hours*3600//30:
            break
        
        

In [1]:
# Specify parameters for augmentation

audio_config = {}

audio_config['features_output_dir'] = '/Volumes/MachineLearning/training_data/okay_nabu/generated_positive/training/training_0_mmap'

audio_config['input_path'] = '/Volumes/MachineLearning/audio_samples/background_noise_samples/wav_16k/fsd50k_no_speech'
audio_config['input_glob'] = '*.wav'
audio_config['impulse_paths'] = None#['/Volumes/MachineLearning/audio_samples/background_noise_samples/wav_16k/bird_rir_all_channels_16k']
audio_config['background_paths'] = None#['/Volumes/MachineLearning/audio_samples/background_noise_samples/background_clips_for_mixing']
audio_config['min_clip_duration_s'] = 5.0
audio_config['max_clip_duration_s'] = None
audio_config['max_start_time_from_right_s'] = None
audio_config['max_jitter_s'] = None
audio_config['augmented_duration_s'] = None

from microwakeword.feature_generation import ClipsHandler

In [2]:
# Load audio clips and prepare them for augmentation

clips_handler = ClipsHandler(
                            input_path=audio_config['input_path'],
                            input_glob=audio_config['input_glob'],
                            impulse_paths=audio_config['impulse_paths'], 
                            background_paths=audio_config['background_paths'], 
                            augmentation_probabilities = {
                                "SevenBandParametricEQ": 0.0,
                                "TanhDistortion": 0.0,
                                "PitchShift": 0.0,
                                "BandStopFilter": 0.0,
                                "AddBackgroundNoise": 0.0,
                                "Gain": 0.0,
                                "RIR": 0.0,
                            },
                            augmented_duration_s = audio_config['augmented_duration_s'],
                            max_start_time_from_right_s = audio_config['max_start_time_from_right_s'],
                            max_jitter_s = audio_config['max_jitter_s'],
                            max_clip_duration_s = audio_config['max_clip_duration_s'],   
                            min_clip_duration_s = audio_config['min_clip_duration_s'],     
)

In [6]:
# Test by playing a randomly augmented clip

import IPython

clips_handler.save_random_augmented_clip("augmented_clip.wav")

IPython.display.display(IPython.display.Audio("augmented_clip.wav"))

In [7]:
# Save all the clip's augmented features in a Ragged Mmap
for i in range(0,1):
    # audio_config['features_output_dir'] = '/Volumes/MachineLearning/training_data/hey_mycroft_hand_filtered/training/training_{}_mmap'.format(str(i))
    # print(audio_config['features_output_dir'])
    clips_handler.save_augmented_features(audio_config['features_output_dir'])

0it [00:00, ?it/s]

2024-03-19 11:48:08.111385: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-03-19 11:48:08.111419: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-03-19 11:48:08.111429: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-03-19 11:48:08.111633: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-19 11:48:08.111895: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
15it [00:00, 16.65it/s]UserWarning:
491it [00:11, 46.91it/s]UserWarning:
    The file /Volumes/MachineLearning/audio_samples/background_noise_samples/background_clips_for_mixing/fsd

In [31]:
# Save all the clip's augmented features in a Ragged Mmap
for i in range(0,1):
    # audio_config['features_output_dir'] = '/Volumes/MachineLearning/training_data/hey_mycroft_hand_filtered/training/training_{}_mmap'.format(str(i))
    # print(audio_config['features_output_dir'])
    clips_handler.save_augmented_features(audio_config['features_output_dir'])

2130it [00:48, 40.90it/s]UserWarning:
    The file /Volumes/MachineLearning/audio_samples/background_noise_samples/background_clips_for_mixing/fsd50k/83748.wav.wav is too silent to be added as noise. Returning the input unchanged.
25981it [10:09, 37.80it/s]UserWarning:
    The file /Volumes/MachineLearning/audio_samples/background_noise_samples/background_clips_for_mixing/fsd50k/344558.wav.wav is too silent to be added as noise. Returning the input unchanged.
35410it [13:48, 41.75it/s]UserWarning:
    The file /Volumes/MachineLearning/audio_samples/background_noise_samples/background_clips_for_mixing/fsd50k/427296.wav.wav is too silent to be added as noise. Returning the input unchanged.
36147it [14:05, 51.37it/s]UserWarning:
    The file /Volumes/MachineLearning/audio_samples/background_noise_samples/background_clips_for_mixing/fsd50k/343069.wav.wav is too silent to be added as noise. Returning the input unchanged.
37066it [14:28, 38.74it/s]UserWarning:
    The file /Volumes/MachineLe

In [6]:
# Specify parameters for augmentation

audio_config = {}

audio_config['features_output_dir'] = '/Volumes/MachineLearning/training_data/okay_nabu/generated_positive/training/training_0_mmap'

audio_config['input_path'] = '/Volumes/MachineLearning/multilingual_stuff/en/clips/abraham'
audio_config['input_glob'] = '**/*.opus'
audio_config['impulse_paths'] = ['/Volumes/MachineLearning/audio_samples/background_noise_samples/wav_16k/bird_rir_all_channels_16k']
audio_config['background_paths'] = ['/Volumes/MachineLearning/audio_samples/background_noise_samples/background_clips_for_mixing']
audio_config['min_clip_duration_s'] = None
audio_config['max_clip_duration_s'] = 1.39
audio_config['max_start_time_from_right_s'] = None
audio_config['max_jitter_s'] = 0.0001
audio_config['augmented_duration_s'] = 5.0

from microwakeword.feature_generation import ClipsHandler

In [7]:
# Load audio clips and prepare them for augmentation

clips_handler = ClipsHandler(
                            input_path=audio_config['input_path'],
                            input_glob=audio_config['input_glob'],
                            impulse_paths=audio_config['impulse_paths'], 
                            background_paths=audio_config['background_paths'], 
                            augmentation_probabilities = {
                                "SevenBandParametricEQ": 0.25,
                                "TanhDistortion": 0.25,
                                "PitchShift": 0.25,
                                "BandStopFilter": 0.25,
                                "AddBackgroundNoise": 1,
                                "Gain": 1.0,
                                "RIR": 0.0,
                            },
                            augmented_duration_s = audio_config['augmented_duration_s'],
                            max_start_time_from_right_s = audio_config['max_start_time_from_right_s'],
                            max_jitter_s = audio_config['max_jitter_s'],
                            max_clip_duration_s = audio_config['max_clip_duration_s'],   
                            min_clip_duration_s = audio_config['min_clip_duration_s'],
                            remove_silence=True,
                            truncate_clip_s=0.58,
)

In [10]:
# Test by playing a randomly augmented clip

import IPython

clips_handler.save_random_augmented_clip("augmented_clip.wav")

IPython.display.display(IPython.display.Audio("augmented_clip.wav"))